In [4]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import pandas as pd
import pulp
import csv


pandas.set_option('display.width', 1000)

In [5]:
# create dataframe of past salaries and past performance
fantasy_df = pd.read_csv("../Data/predicted2.csv")

# Print the first five rows of data to the screen
fantasy_df.head()

,Player_Name,FD_pos,position,FD_Sal,PFDP,Last_FDP,Rank,pt,rb,as,st,bl,to,trey,FGM,FGA,FTM,FTA
0,Alex_Len,5,C,4400,7.64,16.8,belowAvg,8.725000,5.775000,1.050000,0.50,0.825000,1.000000,0.350,3.300000,6.125000,1.275000,2.025000
1,Al-Farouq_Aminu,4,PF,3800,9.72,12.9,belowAvg,4.333333,4.888889,1.166667,1.00,0.444444,0.944444,0.500,1.388889,4.777778,1.055556,1.611111
2,Alfonzo_McKinnie,3,SF,3700,4.06,7.4,benchPlayer,4.125000,2.843750,0.375000,0.50,0.187500,0.593750,0.375,1.625000,3.906250,0.500000,0.718750
3,Andrew_Wiggins,2,SG,7700,21.56,21.5,average,22.675000,5.150000,3.675000,0.65,0.850000,2.425000,2.200,4.175000,10.375000,3.525000,4.875000
4,Anfernee_Simons,1,PG,3500,12.05,16.5,belowAvg,8.920000,2.480000,1.500000,0.34,0.180000,1.000000,1.080,3.360000,8.320000,1.120000,1.400000


In [81]:
# choose an arbitrary date
# date = 20151116

In [82]:
# isolate data for specific date
# specificDate = pastsalaries[pastsalaries['Date'] == date]

# # exclude players with missing salaries
# specificDate = specificDate[specificDate['DK Salary'] != 'nan']

In [8]:
# initialize variables
playerPositions = fantasy_df['position'].tolist()
playerNames = fantasy_df['Player_Name'].tolist()
playerCosts = tuple(fantasy_df['FD_Sal'])
playerPoints = tuple(fantasy_df['Last_FDP'])
salaryCap = 60000
numOfPlayers = range(len(playerNames))

In [9]:
# formulate linear programming problem
problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)

C:\Users\ddrag\Anaconda3\envs\pythondata\lib\site-packages\pulp\pulp.py:1137: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [10]:
# create variable to represent each player
playerInLineup = pulp.LpVariable.dicts("Players", [i for i in numOfPlayers], 0, 1, cat="Binary")

In [11]:
# create objective: maximize sum of player points
problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)

In [12]:
# constraint: each player can only be chosen at most once
for i in numOfPlayers:
    problem += pulp.lpSum(playerInLineup[i]) <= 1

In [13]:
# constraints: sum of player costs must be less than or equal to the salary cap
problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap

In [14]:
# constraint: teams must have 9 players
problem += pulp.lpSum(problem.variables()) == 9
problem


Optimal_Lineup:
MAXIMIZE
16.8*Players_0 + 12.9*Players_1 + 17.4*Players_10 + 6.6*Players_100 + 13.5*Players_101 + 1.7*Players_102 + 19.7*Players_103 + 33.2*Players_104 + 14.2*Players_105 + 25.2*Players_106 + 37.8*Players_107 + 6.9*Players_108 + 29.0*Players_109 + 6.2*Players_11 + 1.2*Players_110 + 21.1*Players_111 + 7.0*Players_112 + -1.0*Players_113 + 9.7*Players_114 + 4.0*Players_115 + 19.7*Players_116 + 20.5*Players_117 + 4.0*Players_118 + 16.7*Players_119 + 27.9*Players_12 + 19.0*Players_120 + 2.7*Players_121 + 12.9*Players_122 + 19.0*Players_123 + 4.6*Players_124 + 6.2*Players_125 + 17.2*Players_126 + 2.0*Players_127 + 13.1*Players_129 + 34.5*Players_13 + 37.5*Players_131 + 3.0*Players_14 + 22.6*Players_15 + 2.0*Players_16 + 7.5*Players_17 + 22.5*Players_18 + 3.2*Players_19 + 7.4*Players_2 + 49.2*Players_20 + 8.9*Players_21 + 21.7*Players_22 + 7.8*Players_23 + 28.8*Players_25 + 12.4*Players_26 + 34.7*Players_27 + 33.7*Players_28 + 12.7*Players_29 + 21.5*Players_3 + 34.0*Players_30

#Point Guards

In [17]:
avail_PG = 2
total_PG = ""
for rownum, row in fantasy_df.iterrows():
    for i, player in enumerate(numOfPlayers):
        if rownum == i:
            if row['FD_pos'] == 'PG':
                formula = 1*player
                total_PG += formula
problem += (total_PG == avail_PG)
print(total_PG)

C:\Users\ddrag\Anaconda3\envs\pythondata\lib\site-packages\pulp\pulp.py:1385: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [19]:
# constraint: number of PG = 2
pointguards = tuple(fantasy_df['position'] == 'PG')
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) <= 2

In [21]:
# constraint: number of SG = 2
shootinguard = tuple(fantasy_df['position'] == 'SG')
problem += pulp.lpSum(playerInLineup[i] * shootinguard[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * shootinguard[i] for i in numOfPlayers) <= 2

In [22]:
# constraint: number of SF = 2
smallforward = tuple(fantasy_df['position'] == 'SF')
problem += pulp.lpSum(playerInLineup[i] * smallforward[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * smallforward[i] for i in numOfPlayers) <= 2

In [23]:
# constraint: number of PF = 2
powerforward = tuple(fantasy_df['position'] == 'PF')
problem += pulp.lpSum(playerInLineup[i] * powerforward[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * powerforward[i] for i in numOfPlayers) <= 2

In [24]:
# constraint: number of C = 1
center = tuple(fantasy_df['position'] == 'C')
problem += pulp.lpSum(playerInLineup[i] * center[i] for i in numOfPlayers) >=1
problem += pulp.lpSum(playerInLineup[i] * center[i] for i in numOfPlayers) <= 1

In [28]:
# calculated predicted lineup
cost = 0
points = 0

predicted = set()
if problem.solve() == 1:
    for pos in range(len(numOfPlayers)):
        if playerInLineup[pos].value() == 1:
            name = playerNames[pos]
            predicted.add(name)
            cost += playerCosts[pos]
            points += playerPoints[pos]
            print (playerNames[pos], playerPositions[pos], playerCosts[pos],playerPoints[pos])
    print (cost, points)
else:
    print ('Error finding solution')

Ben_Simmons PG 9300 40.3
De'Aaron_Fox PG 7900 34.7
Drew_Eubanks C 3500 2.0
Harry_Giles PF 3800 26.1
Jrue_Holiday SG 7800 22.6
LeBron_James SF 11000 47.5
Marcus_Morris SF 5700 36.2
Royce_O'Neale PF 3900 19.7
Victor_Oladipo SG 5500 12.9
58400 241.99999999999997


In [29]:
#IDEAL VS PREDICTED?